In [1]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
import time

In [2]:
def check_symbol (stock_df ,label) :
    test_dr = pd.DataFrame()
    for i in range (len(stock_df)):
        if len(stock_df.iloc[i][label]) == 4:
            test_dr = test_dr.append(stock_df[i:i+1]).reset_index(drop=True)
    return test_dr

In [3]:
def extract(Stock_list, whole_df , label):
    start = 0
    end = 0
    point = Stock_list[0]
    for i in range (len(whole_df)):
        if whole_df.iloc[i][label] >= str(point):
            #print (whole_df.iloc[i][label] , i)
            if start != 0 :
                end = i
                break
            point = Stock_list[len(Stock_list)-1]
            start = i
    test =  pd.DataFrame(whole_df[start:end+1])
    return test

In [4]:
def find_index(Stock_list, extract, label,  null) :
    institutional = pd.DataFrame()
    for i in range (len(Stock_list)):
        a = extract[extract[label] == str(Stock_list[i])].index
        if len(a)== 1  :
            institutional=institutional.append(extract.iloc[a])
        elif len(a) == 0 :
            institutional=institutional.append(null)
    return institutional.reset_index(drop=True)

In [5]:
def devide(list1 , list2) :
    out_come = list()
    for i in range (len(list1)):
        if type (list1[i]) != float and  int(list2[i].replace(",", "")) >0 :
            x = format(int(list1[i].replace(",", "")) / int(list2[i].replace(",", "")) ,'.3f')
        else :
            x = 0
        out_come.append(x)
    return out_come

In [6]:
Stock_list = pd.read_csv('name.csv')["name"]
date_list = pd.read_csv('date.csv')['date'].tolist()
whole_data = pd.DataFrame()
Stock_index = pd.read_csv('Market_index.csv')

i=0
today = datetime.date.today()
day = str (today-datetime.timedelta(days=i+553)).replace("-", "")
day

In [7]:
# generate date
i = 0
today = datetime.date.today()
day = str (today-datetime.timedelta(days=i)).replace("-", "")
record_day =  max(date_list) 
sleep_download_time = 3

while int(day) > record_day  :
#while i < 4000  :
    # get basic data
    
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + day + '&type=ALL')
    if len(r.text) < 100 :
        print (day ," is not work day!")
        time.sleep(sleep_download_time)
        i+=1
        day = str (today-datetime.timedelta(days=i)).replace("-", "")
        continue
        
    label_ = "證券代號"
    df = pd.read_csv(StringIO(r.text.replace("=", "")),  header=["證券代號" in l for l in r.text.split("\n")].index(True)-1).drop(['Unnamed: 16'],axis = 1)
    df = df.dropna().reset_index(drop=True)
    time.sleep(sleep_download_time)
    
    # process the basic data by name list
    base_df = extract(Stock_list , df , label_).reset_index(drop=True)
    stock_df = find_index(Stock_list, base_df, label_,  df[-1:])
    
    ## get institution data
    r = requests.get('https://www.twse.com.tw/fund/T86?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df2 = pd.read_csv(StringIO(r.replace("  \n", "")),  header=["證券代號" in l for l in r.split("\r\n")].index(True))
    df2.sort_values(by='證券代號' , ascending= True, axis =0 , inplace=True)
    df2 = df2.reset_index(drop=True).drop(['Unnamed: 19'],axis = 1)
    
    institutional = extract(Stock_list , df2 , label_).reset_index(drop=True)
    institutional = find_index(Stock_list, institutional, label_,  df2[-1:])
    time.sleep(sleep_download_time)
    ## process margin data
    r = requests.get('https://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date=' + day + '&selectType=ALL')
    r = r.text.replace("=", "")
    df3 = pd.read_csv(StringIO(r), header=["股票代號" in l for l in r.split("\n")].index(True)).drop(['Unnamed: 16'],axis = 1)

    label_ = "股票代號"
    margin = extract(Stock_list, df3 , label_).reset_index(drop=True)
    margin = find_index(Stock_list, margin, label_,  df3[-1:])
    ratio = devide(margin['今日餘額.1'], margin['今日餘額'] )
    margin ['ratio'] = ratio 
    time.sleep(sleep_download_time)
    
    # merge to basic df
    stock_df['融資買'] = margin['買進']
    stock_df['融資賣'] = margin['賣出']
    stock_df['融資餘'] = margin['今日餘額']
    stock_df['融卷買'] = margin['買進.1']
    stock_df['融卷賣'] = margin['賣出.1']
    stock_df['融卷餘'] = margin['今日餘額.1']
    stock_df['ratio'] = margin ['ratio']

    stock_df['外資進出'] = institutional['外陸資買賣超股數(不含外資自營商)']
    stock_df['外資進'] = institutional['外陸資買進股數(不含外資自營商)']
    stock_df['外資出'] = institutional['外陸資賣出股數(不含外資自營商)']
    stock_df['投信進出'] = institutional['投信買賣超股數']
    stock_df['投信進'] = institutional['投信買進股數']
    stock_df['投信出'] = institutional['投信賣出股數']
    stock_df['自營進出'] = institutional['自營商買賣超股數(自行買賣)']
    stock_df['自營進'] = institutional['自營商買進股數(自行買賣)']
    stock_df['自營出'] = institutional['自營商賣出股數(自行買賣)']
    stock_df['避險進出'] = institutional['自營商買賣超股數(避險)']
    stock_df['避險進'] = institutional['自營商買進股數(避險)']
    stock_df['避險出'] = institutional['自營商賣出股數(避險)']
    stock_df['三大'] = institutional['三大法人買賣超股數']
    
    #stock_df  = stock_df.dropna().reset_index(drop=True)
    
    ## add date label
    date = [day for i in range(len(stock_df))]
    stock_df ["date"] = date
    whole_data = whole_data.append(stock_df)
    date_list.append(int(day))
    
    # for market index
    index_set = ['發行量加權股價指數' , '未含電子指數' , '電子類兩倍槓桿指數' ]
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date='+ day +'&type=IND')
    df = pd.read_csv(StringIO(r.text.replace("價格指數(臺灣證券交易所)\"\r\n", "")),  header=["指數" in l for l in r.text.split("\r\n")].index(True)).drop(['漲跌(+/-)','特殊處理註記','Unnamed: 6'],axis = 1)
    df1 = df.dropna()
    df1.columns = ['Index','收盤','漲跌','漲跌%']
    index = pd.DataFrame()
    for x in index_set :
        index = index.append(df1[df1.Index == x])
    index['date'] = [int(day) for x in range (len(index))]
    Stock_index = Stock_index.append(index)
    time.sleep(sleep_download_time)    
    print (day)
    
    i+=1
    day = str (today-datetime.timedelta(days=i)).replace("-", "")

20210416  is not work day!
20210415


In [8]:
date = pd.DataFrame()
date['date'] = date_list
date = date.drop_duplicates()
date = date.sort_values(by= 'date',ascending=False)
date.to_csv('date.csv', index = False)

In [9]:
date

,date
525,20210415
0,20210414
1,20210413
2,20210412
3,20210409
...,...
520,20190221
521,20190220
522,20190219
523,20190218


In [10]:
drop = whole_data.dropna().reset_index(drop=True)
whole_data = drop.drop(['漲跌(+/-)', '證券名稱' ,'最後揭示買價', '最後揭示買量', '最後揭示賣價', '最後揭示賣量'],axis = 1)

In [11]:
for l in whole_data.columns : 
    list_ = []
    for i in range(len(whole_data)) :
        k = whole_data[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    whole_data[l] = list_

In [12]:
Stock_index = Stock_index.drop_duplicates()
Stock_index = Stock_index.reset_index(drop=True)
Stock_index

,Index,收盤,漲跌,漲跌%,date
0,發行量加權股價指數,16211.7,264.85,1.66,20210303
1,未含電子指數,16115,273.49,1.73,20210303
2,電子類兩倍槓桿指數,40860.8,1287.77,3.25,20210303
3,發行量加權股價指數,15946.9,6.92,-0.04,20210302
4,未含電子指數,15841.5,60.53,-0.38,20210302
...,...,...,...,...,...
1523,未含電子指數,17633.2,51.06,0.29,20210414
1524,電子類兩倍槓桿指數,41811,182.26,0.44,20210414
1525,發行量加權股價指數,"17,076.73",210.76,1.25,20210415
1526,未含電子指數,"17,911.66",278.45,1.58,20210415


In [13]:
for l in ['收盤','漲跌','漲跌%'] : 
    list_ = []
    for i in range(len(Stock_index)) :
        k = Stock_index[l][i]
        if type(k) == str :
            try :
                list_.append(float (k.replace(",", "")))
            except:
                print(l , i , k)
                list_.append(0)
                continue
        else : 
            list_.append(float(k))
    Stock_index[l] = list_

In [14]:
whole_data = whole_data [whole_data['開盤價'] > 0]
whole_data

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,20865248.0,8288.0,1.002854e+09,47.40,49.30,47.40,48.30,0.90,11.03,...,1000000.0,448000.0,175640.0,488640.0,313000.0,79000.0,140000.0,61000.0,-1854520.0,20210415.0
1,1103.0,5474550.0,1858.0,1.250991e+08,22.45,23.00,22.45,22.95,0.55,8.38,...,0.0,0.0,5000.0,6000.0,1000.0,0.0,0.0,0.0,2089000.0,20210415.0
2,1104.0,8372843.0,4001.0,2.158739e+08,25.35,26.35,25.00,26.10,0.75,13.66,...,173000.0,0.0,4000.0,4000.0,0.0,0.0,0.0,0.0,-705000.0,20210415.0
3,1108.0,15613549.0,5688.0,2.284503e+08,14.00,15.10,13.95,14.70,0.85,15.15,...,0.0,0.0,3000.0,3000.0,0.0,0.0,0.0,0.0,-1097100.0,20210415.0
4,1109.0,1669743.0,637.0,4.378872e+07,25.90,26.45,25.75,26.35,0.50,8.96,...,0.0,0.0,-248002.0,3000.0,251002.0,0.0,0.0,0.0,182998.0,20210415.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,9944.0,216226.0,300.0,4.362526e+06,20.10,20.30,20.00,20.30,0.30,29.42,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77000.0,20210415.0
907,9945.0,5965520.0,2717.0,2.983822e+08,49.65,50.30,49.55,50.10,0.55,8.77,...,0.0,0.0,9000.0,12000.0,3000.0,-50000.0,33000.0,83000.0,1190676.0,20210415.0
908,9946.0,339040.0,223.0,5.526208e+06,16.35,16.45,16.15,16.45,0.05,28.86,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-58000.0,20210415.0
909,9955.0,287381.0,360.0,5.180289e+06,17.85,18.25,17.80,18.20,0.50,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67000.0,20210415.0


In [15]:
previous = pd.read_csv('clear.csv')
previous = previous.drop_duplicates()
previous 

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,11924201.0,4904.0,565071302.0,47.40,47.75,47.05,47.40,0.05,10.82,...,2666000.0,0.0,43000.0,65000.0,22000.0,45000.0,65000.0,20000.0,700942.0,20210414.0
1,1103.0,5039420.0,1790.0,112232457.0,22.85,22.85,21.70,22.40,0.35,8.18,...,0.0,0.0,2000.0,2000.0,0.0,0.0,0.0,0.0,632001.0,20210414.0
2,1104.0,9421945.0,3862.0,236318631.0,25.25,25.65,24.20,25.35,0.15,13.27,...,0.0,0.0,31000.0,32000.0,1000.0,0.0,0.0,0.0,3750.0,20210414.0
3,1108.0,3677239.0,1528.0,50220578.0,13.90,14.00,13.30,13.85,0.05,14.28,...,0.0,0.0,1000.0,1000.0,0.0,0.0,0.0,0.0,773000.0,20210414.0
4,1109.0,2543529.0,1063.0,65744287.0,26.50,26.70,25.15,25.85,0.40,8.79,...,0.0,0.0,-48999.0,1002.0,50001.0,0.0,0.0,0.0,-67999.0,20210414.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446988,9943.0,190438.0,168.0,12812017.0,67.50,67.50,67.20,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
446989,9945.0,2752245.0,1096.0,116365981.0,42.60,42.60,42.10,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
446990,9946.0,1184358.0,469.0,27781296.0,23.50,23.60,23.30,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
446991,9955.0,538156.0,334.0,10280286.0,18.80,19.20,18.80,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [16]:
K = whole_data.append(previous).reset_index(drop=True)
K

,證券代號,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,本益比,...,投信進,投信出,自營進出,自營進,自營出,避險進出,避險進,避險出,三大,date
0,1102.0,20865248.0,8288.0,1.002854e+09,47.40,49.30,47.40,48.30,0.90,11.03,...,1000000.0,448000.0,175640.0,488640.0,313000.0,79000.0,140000.0,61000.0,-1854520.0,20210415.0
1,1103.0,5474550.0,1858.0,1.250991e+08,22.45,23.00,22.45,22.95,0.55,8.38,...,0.0,0.0,5000.0,6000.0,1000.0,0.0,0.0,0.0,2089000.0,20210415.0
2,1104.0,8372843.0,4001.0,2.158739e+08,25.35,26.35,25.00,26.10,0.75,13.66,...,173000.0,0.0,4000.0,4000.0,0.0,0.0,0.0,0.0,-705000.0,20210415.0
3,1108.0,15613549.0,5688.0,2.284503e+08,14.00,15.10,13.95,14.70,0.85,15.15,...,0.0,0.0,3000.0,3000.0,0.0,0.0,0.0,0.0,-1097100.0,20210415.0
4,1109.0,1669743.0,637.0,4.378872e+07,25.90,26.45,25.75,26.35,0.50,8.96,...,0.0,0.0,-248002.0,3000.0,251002.0,0.0,0.0,0.0,182998.0,20210415.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447899,9943.0,190438.0,168.0,1.281202e+07,67.50,67.50,67.20,67.20,0.20,11.18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-52000.0,20190920.0
447900,9945.0,2752245.0,1096.0,1.163660e+08,42.60,42.60,42.10,42.10,0.50,4.38,...,0.0,81000.0,40000.0,40000.0,0.0,-56000.0,0.0,56000.0,-684265.0,20190920.0
447901,9946.0,1184358.0,469.0,2.778130e+07,23.50,23.60,23.30,23.60,0.10,5.58,...,0.0,0.0,-78000.0,12000.0,90000.0,0.0,0.0,0.0,107000.0,20190920.0
447902,9955.0,538156.0,334.0,1.028029e+07,18.80,19.20,18.80,19.15,0.35,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44000.0,20190920.0


In [17]:
K.columns

Index(['證券代號', '成交股數', '成交筆數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差',
       '本益比', '融資買', '融資賣', '融資餘', '融卷買', '融卷賣', '融卷餘', 'ratio', '外資進出', '外資進',
       '外資出', '投信進出', '投信進', '投信出', '自營進出', '自營進', '自營出', '避險進出', '避險進', '避險出',
       '三大', 'date'],
      dtype='object')

In [18]:
K.to_csv('clear.csv', index = False)
Stock_index.to_csv('Market_index.csv', index = False)